<!--NOTEBOOK_HEADER-->
*This notebook contains course material from [CBE40455](https://jckantor.github.io/CBE40455) by
Jeffrey Kantor (jeff at nd.edu); the content is available [on Github](https://github.com/jckantor/CBE40455.git).
The text is released under the [CC-BY-NC-ND-4.0 license](https://creativecommons.org/licenses/by-nc-nd/4.0/legalcode),
and code is released under the [MIT license](https://opensource.org/licenses/MIT).*

<!--NAVIGATION-->
< [Risk Neutral Gambler](http://nbviewer.jupyter.org/github/jckantor/CBE40455/blob/master/notebooks/06.03-Risk-Neutral-Gambler.ipynb) | [Contents](toc.ipynb) | [Points after Touchdown Decision](http://nbviewer.jupyter.org/github/jckantor/CBE40455/blob/master/notebooks/06.05-Points-after-Touchdown-Decision.ipynb) ><p><a href="https://colab.research.google.com/github/jckantor/CBE40455/blob/master/notebooks/06.04-Risk-Averse-Gambler.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://raw.githubusercontent.com/jckantor/CBE40455/master/notebooks/06.04-Risk-Averse-Gambler.ipynb"><img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# Risk Averse Gambler

## Problem Statement

A risk averse gambler with a stake of money enters a game with the idea of betting for a fixed number of rounds $T$. With a stake $x$ and wager $u$, the resulting state is either $x+u$ with probability $p$, or $x-u$ with probability $q$ (where $p + q \leq 1$.) The wager must be an integer smaller than the current stake or the maximum wager established for the game. The total stake is limited to an amount $N$. The gambler is risk averse where utility of the final stake is $\log(x)$. Given an initial stake $x \lt N$, calculate a strategy that maximizes the expected utility at the end of the game.

## Solution by Stochastic Dynamic Programming

The function $V(k,x)$ is the expected utility after of stake $x$ after the $k^{th}$ wager. The expected utility satisfies the optimality equation 

$$V(k,x) = \max_u [ p V(k+1,x+u) + q V(k+1,x-u) ]$$ 

where $V(T,x) = \log(x)$. The maximization is over the set of possible bets ranging from $0$ to the minimum of $x$, $N-x$, or the bet limit $B$. Note that the state space and set of control actions are finite.

## Solution by Linear Programming

The optimality equation can be solved by well known methods for policy iteration.  Alternatively, as shown for example by Ross in "Introduction to Stochastic Dynamic Programming" (Academic Press, 1983), an exact solution can be found by linear programming. We seek a solution $V[k,x]$ minimizing 

$$\sum_{k=0}^{T-1}\sum_{x=0}^N V[k,x]$$

subject to 
    
$$V[k,x] \geq p V[k+1,x+u] + q V[k+1,x-u]$$

for all feasible bets and boundary condition $V[T,x] = \log(x)$. The set of optimal wagers $u[x]$ are found by determining the constraints that are active at optimality.

## MathProg Model

In [3]:
%%script glpsol -m /dev/stdin

/* Example:  RAGambling.mod Stochastic Dynamic Programming: Risk Averse Gambler */

/* Problem Parameters.  Any of these can be adjusted in a data section.  */

param T default 5 >= 1;              # Stages
param N default 50, >= 1;            # Maximum Stake (reduce if computations are slow)
param p default 0.55, >= 0, <= 1;    # Winning probability
param q default 1-p, >= 0, <= 1-p;   # Losing probability
param B default N, >= 1, <= N;       # Maximum wager size

/* Set of States */

set X:= 1..N;

/* Sets of possible wagers. These are parameterized by the State */

set U{x in X} := 0..min(B,min(N-x,x-1));

/* Value function */

var V{0..T,X}>=0;

/* Exact Linear Program Equivalent of the DP */

minimize OBJ: sum{t in 0..T-1, x in X} V[t,x] ;

s.t. C1 {t in 0..T-1, x in 1..N, u in U[x]}:
   V[t,x] >=  p*V[t+1,x+u] + q*V[t+1,x-u];
s.t. C2 {x in X}: V[T,x] = log(x);

solve;

/* Find Optimal Wager */
param w{t in 0..T-1,x in 1..N} := 
   if x==N then 0
   else min{u in U[x]:
      abs(-V[t,x]+p*V[t+1,x+u]+q*V[t+1,x-u])<0.000001} u;

#table tab1 {x in X} OUT "JSON" "Optimal Wager" "LineChart" : 
#    x, w[T-1,x]~Wager;
    
#table tab2 {x in X} OUT "JSON" "Expected Utility of the Initial Stake" "LineChart" :
#    x, exp(V[T-1,x])~ExpectedUtility;

printf "   Number of Wagers = %4d\n", T;
printf "      Maximum Stake = %4d\n", N;
printf "        Maximum Bet = %4d\n", B;
printf "Winning Probability = %8.3f\n", p ;
printf " Losing Probability = %8.3f\n", q ;
printf "\n  %7s ",' ';
printf {t in 0..T-1} "   Wager %2s  ", t+1;
printf "\n %7s ",'Stake';
printf {t in 0..T-1} "   CE[x] u[x]";
printf "\n %7s ",'-----';
printf {t in 0..T-1} "   %9s ", '---------';
for {x in X}{
   printf "\n %7d", x;
   printf {t in 0..T-1} "   %6.2f %3d", exp(V[t,x]), w[t,x];
}
printf "\n";

end;

GLPSOL: GLPK LP/MIP Solver, v4.52
Parameter(s) specified in the command line:
 -m /dev/stdin
Reading model section from /dev/stdin...
/dev/stdin:66: warning: final NL missing before end of file
66 lines were read
Generating OBJ...
Generating C1...
Generating C2...
Model has been successfully generated
GLPK Simplex Optimizer, v4.52
3301 rows, 300 columns, 9800 non-zeros
Preprocessing...
2598 rows, 248 columns, 7596 non-zeros
Scaling...
 A: min|aij| =  4.500e-01  max|aij| =  1.000e+00  ratio =  2.222e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2598
      0: obj =   1.486763158e+02  infeas =  1.962e+03 (0)
    500: obj =   4.798199905e+02  infeas =  6.426e+02 (0)
   1000: obj =   5.925871650e+02  infeas =  2.744e+02 (0)
   1500: obj =   7.381673968e+02  infeas =  3.447e+01 (0)
*  1630: obj =   7.602764649e+02  infeas =  1.050e-15 (0)
*  1962: obj =   7.453389547e+02  infeas =  6.439e-16 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.2 sec

<!--NAVIGATION-->
< [Risk Neutral Gambler](http://nbviewer.jupyter.org/github/jckantor/CBE40455/blob/master/notebooks/06.03-Risk-Neutral-Gambler.ipynb) | [Contents](toc.ipynb) | [Points after Touchdown Decision](http://nbviewer.jupyter.org/github/jckantor/CBE40455/blob/master/notebooks/06.05-Points-after-Touchdown-Decision.ipynb) ><p><a href="https://colab.research.google.com/github/jckantor/CBE40455/blob/master/notebooks/06.04-Risk-Averse-Gambler.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://raw.githubusercontent.com/jckantor/CBE40455/master/notebooks/06.04-Risk-Averse-Gambler.ipynb"><img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>